In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [80]:
# vanilla RSA adapted from seminar's RSA model
## My changes:
## Changed constructor to include truth_matrix
## Removed L0_truth_table() (now included in constructor)
## Made model more flexible, i.e. less specific to the previous assignment

## note to self: barebones for now

class RSAModel():
    def __init__(self, objects, messages, truth_table, alpha, priors, cost_function):
        if np.shape(truth_table) != (len(messages),len(objects)):
          raise ValueError("""Truth matrix must be m x n, 
          where m is number of messages and n is the number of objects""")
        self.objects = objects
        self.messages = messages
        self.truth_table = truth_table
        self.alpha = alpha
        self.priors = priors
        self.cost_function = cost_function
        
        # the following are for convenience
        self.n_obj = len(self.objects)
        self.n_mes = len(self.messages)
    
    def __normalize(self, arr):
        return arr / arr.sum(axis=1)[:, np.newaxis]
    
    def L0(self):
        #compute [[m]](o): literal meaning
        literal_meaning = self.truth_table

        #P(o) -- create a matrix of priors by repeating the priors array for each row
        priors = np.tile(np.array(self.priors), (self.n_mes,1))

        #multiply the two of L0 terms together
        unnorm  =  literal_meaning * priors

        #normalize to obtain a probability distribution
        norm = self.__normalize(unnorm)

        return norm
    
    def S1(self):
        #compute the costs for each expression in self.messages
        costs = self.cost_function()

        #utility without costs
        epsilon = 1e-10 # to prevent log(0)
        raw_utility = np.log(self.L0().T + epsilon)

        #Cost(m) -- reshape to obtain correct dimensions for matrix multiplication
        costs = np.repeat(self.cost_function(),2).reshape((self.n_mes, self.n_obj)).T # may be an error here

        #compute utility by subtracting costs from informativity
        utility = np.subtract(raw_utility,costs)

        #compute S1(m|o) by taking the exponent of utility times the temperature parameter alpha
        unnorm = np.exp(self.alpha*utility)

        #normalize to obtain a probability distribution
        norm = self.__normalize(unnorm)

        return norm
    
    def L1(self):
        #S1(m|o)
        S1 = self.S1()
        
        #P(o) -- create a matrix of priors by repeating the priors array for each row
        priors = np.tile(np.array(self.priors), (self.n_obj, 1)) # may be an error here

        #multiply S1(m|o) (transformed, since we need a distribution over objects, not over messages) and P(o) together
        unnorm = S1.T * priors

        #normalize to obtain a probability distribution
        norm = self.normalize(unnorm)

        return norm       

In [81]:
# Example of vanilla model
objects = ['greyhound','dalmatian']
messages = ['dog','greyhound','dalmatian']
truth_table = np.array([[1,1,0],[1,0,1]]).T
print(truth_table.shape)
priors = [0.5,0.5] # i.e. uniform
alpha = 0.8

def cost_function(): # we'll need to figure this out later
    return [0,0,0]

vanilla = RSAModel(objects, messages, truth_table, alpha, priors,cost_function)

(3, 2)


In [82]:
# vanilla L0
vanilla.L0()

array([[0.5, 0.5],
       [1. , 0. ],
       [0. , 1. ]])

In [35]:
# vanilla S1
vanilla.S1()

ValueError: operands could not be broadcast together with shapes (3,2) (2,3) 

In [38]:
np.repeat(vanilla.cost_function(),2).reshape((vanilla.n_mes, vanilla.n_obj))#.T

array([[0, 0],
       [0, 0],
       [0, 0]])

In [43]:
np.tile(np.array(priors), (len(objects),1))

array([[0.334, 0.333, 0.333],
       [0.334, 0.333, 0.333]])